In [1]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [2]:
housing = fetch_california_housing()
m, n = housing.data.shape
print(m, n)

(20640, 8)


In [3]:
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

#normalize input features
scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

In [10]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")

#y(predictions) = theta.X
y_pred = tf.matmul(X, theta, name="predictions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

#Batch gradient descent pg 114
#gradients = 2/m * tf.matmul(tf.transpose(X), error)
gradients = tf.gradients(mse, [theta])[0]


In [11]:
n_epochs = 1000
learning_rate = 0.01

In [12]:
training_op = tf.assign(theta, theta - learning_rate * gradients)

# batch gradient descent step, ø(next step) = ø - MSE(ø)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE=", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()


('Epoch', 0, 'MSE=', 9.790781)
('Epoch', 100, 'MSE=', 5.0945902)
('Epoch', 200, 'MSE=', 4.9422216)
('Epoch', 300, 'MSE=', 4.9041786)
('Epoch', 400, 'MSE=', 4.8795738)
('Epoch', 500, 'MSE=', 4.8614283)
('Epoch', 600, 'MSE=', 4.8478384)
('Epoch', 700, 'MSE=', 4.8375926)
('Epoch', 800, 'MSE=', 4.8298421)
('Epoch', 900, 'MSE=', 4.8239455)


In [13]:
# TensorBoard Visualizations
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

mse_summary = tf.summary.scalar("MSE", mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

